# Fine Tune CLIP on Tweets

- CLIP on huggingface: https://huggingface.co/openai/clip-vit-base-patch32
- Dataset: https://huggingface.co/datasets/AlekseyDorkin/extended_tweet_emojis/tree/main

## 1. Install Dependencies

In [2]:
from IPython import get_ipython

# you might want to restart the kernel
# coupling between torch and torchvision: https://pypi.org/project/torchvision/
get_ipython().system('pip install torchvision==0.11.1 torch==1.10.0 --quiet')
get_ipython().system('pip install transformers datasets pillow ipywidgets requests jupyter jupyter_client wandb sklearn --upgrade --quiet')



## 2. Init Variables and Tools

In [3]:

# pwd = get_ipython().getoutput('pwd')
data_path = "./emojis"
data_path

'./emojis'

In [4]:


import wandb
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "false"
wandb.init(project="emoji-predictor", entity="drift-ai")




/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Curren

## 3. Setup Data Preprocessors and Trainer

In [5]:


from pathlib import Path

import torch
from torchvision.transforms import Resize, InterpolationMode, ConvertImageDtype, CenterCrop
from torchvision.io import read_image, ImageReadMode
from transformers import CLIPProcessor, CLIPModel, Trainer
from transformers import default_data_collator, TrainingArguments
from datasets import load_dataset, Dataset

# Loading Data
dataset = load_dataset("vincentclaes/emoji-predictor")
train_dataset = dataset["train"].select(range(32))
val_dataset = dataset["validation"].select(range(32))
test_dataset = dataset["test"]

column_names = train_dataset.column_names
assert "label" in column_names
assert "text" in column_names
image_column = "label"
caption_column = "text"

Using custom data configuration vincentclaes--emoji-predictor-84ee9ecf6ec78809
Reusing dataset parquet (/root/.cache/huggingface/datasets/vincentclaes___parquet/vincentclaes--emoji-predictor-84ee9ecf6ec78809/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:


# Loading Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
config = model.config
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = processor.tokenizer
feature_extractor = processor.feature_extractor

MAX_TEXT_LENGTH=77
IMAGE_SIZE = config.vision_config.image_size

# Preprocess Text
def tokenize_captions(examples):
    captions = [caption for caption in examples[caption_column]]
    text_inputs = tokenizer(captions, max_length=MAX_TEXT_LENGTH, padding="max_length", truncation=True)
    examples["input_ids"] = text_inputs.input_ids
    examples["attention_mask"] = text_inputs.attention_mask
    return examples


train_dataset = train_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on train dataset",
)

val_dataset = val_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on val dataset",
)

test_dataset = test_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on test dataset",
)
#
# # Preprocess Images
# class Transform(torch.nn.Module):
#     def __init__(self, image_size):
#         super().__init__()
#         self.transforms = torch.nn.Sequential(
#             # resize and then crop the image to the image_size
#             Resize([image_size], interpolation=InterpolationMode.BICUBIC),
#             CenterCrop(image_size),
#             # convert RGB to floats
#             ConvertImageDtype(torch.float),
#         )
#
#     def forward(self, x) -> torch.Tensor:
#         with torch.no_grad():
#             x = self.transforms(x)
#         return x
#
#
# image_transformations = Transform(
#     IMAGE_SIZE
# )
# image_transformations = torch.jit.script(image_transformations)


from PIL import Image

def transform_images(examples):
    # https://pytorch.org/vision/stable/_modules/torchvision/io/image.html#ImageReadMode
    images = [Image.open(str(Path(data_path,f"{c}.png"))) for c in examples[image_column]]
    images_transformed = processor.feature_extractor(images, return_tensors="pt")

    # images = [read_image(str(Path(data_path,f"{c}.png")), ImageReadMode.RGB) for c in examples[image_column]]
    examples["pixel_values"] = images_transformed["pixel_values"]
    return examples


train_dataset.set_transform(transform_images)
val_dataset.set_transform(transform_images)
test_dataset.set_transform(transform_images)


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    input_ids = torch.tensor([example["input_ids"] for example in examples], dtype=torch.long)
    attention_mask = torch.tensor([example["attention_mask"] for example in examples], dtype=torch.long)
    # labels = torch.tensor([example["label"] for example in examples])
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        # "labels": labels,
        "return_loss": True,
    }

# def compute_metrics(p):
#     is_regression = False
#     preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
#     preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
#     if data_args.task_name is not None:
#         result = metric.compute(predictions=preds, references=p.label_ids)
#         if len(result) > 1:
#             result["combined_score"] = np.mean(list(result.values())).item()
#         return result
#     elif is_regression:
#         return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
#     else:
#         return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

from datasets import load_metric
import numpy as np
metric = load_metric("precision")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./checkpoints",
                           dataloader_num_workers=0,
                           per_device_eval_batch_size=8,
                           per_device_train_batch_size=8,
                           num_train_epochs=4,
#                            evaluation_strategy = "steps",
#                            eval_steps=8,
                           warmup_steps=0,
                           learning_rate=5e-05,
                           weight_decay=0.1,
                           report_to="wandb",
                           ),
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    tokenizer=processor
)

Running tokenizer on train dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on val dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

## 4.Train The Model

In [7]:


from transformers.trainer_utils import get_last_checkpoint
# train_result = trainer.train(resume_from_checkpoint=get_last_checkpoint("./checkpoints"))
train_result = trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 32
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/opt/conda/lib/python3.8/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




In [8]:


train_result.metrics

NameError: name 'train_result' is not defined

In [ ]:


trainer.evaluate(ignore_keys=["text_model_output", "vision_model_output", "text_embeds", "logits_per_image"])

In [9]:


trainer.save_model()
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

Saving model checkpoint to ./checkpoints
Configuration saved in ./checkpoints/config.json
Model weights saved in ./checkpoints/pytorch_model.bin
Feature extractor saved in ./checkpoints/preprocessor_config.json
tokenizer config file saved in ./checkpoints/tokenizer_config.json
Special tokens file saved in ./checkpoints/special_tokens_map.json


NameError: name 'train_result' is not defined

In [ ]:


# metrics = trainer.evaluate()
# trainer.log_metrics("eval", metrics)
# trainer.save_metrics("eval", metrics)

In [ ]:

#
# kwargs = {
#     "finetuned_from": "emoji-predictor",
#     "tasks": "contrastive-image-text-modeling",
#     "dataset": "AlekseyDorkin/extended_tweet_emojis"
# }
# trainer.push_to_hub(**kwargs)
trainer